In [10]:
!pip install -q -U PyPDF2
!pip install -q -U spacy

In [11]:
import os
import json
import re
from PyPDF2 import PdfFileReader
import spacy
from collections import defaultdict

In [12]:
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [13]:
nlp = spacy.load("en_core_web_sm")

In [14]:

def extract_text_from_pdf(pdf_path):
    pdf_text = ""
    with open(pdf_path, 'rb') as file:
        reader = PdfFileReader(file)
        for page_num in range(reader.numPages):
            page = reader.getPage(page_num)
            pdf_text += page.extract_text()
    return pdf_text

def clean_text(text):
    # Clean text for better processing
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n+', ' ', text)
    return text.strip()

def split_into_paragraphs(text):
    return [clean_text(p) for p in text.split("\n\n") if p.strip()]

def extract_qa_from_paragraph(paragraph, nlp_model):
    doc = nlp_model(paragraph)
    questions = []
    answers = []
    
    for sent in doc.sents:
        if sent.text.strip().endswith('?'):
            questions.append(sent.text.strip())
        else:
            answers.append(sent.text.strip())
    
    return questions, answers

def create_finetuning_dataset(paragraphs, nlp_model):
    dataset = []
    for para in paragraphs:
        questions, answers = extract_qa_from_paragraph(para, nlp_model)
        for q, a in zip(questions, answers):
            dataset.append({
                "question": q,
                "answer": a,
                "context": para
            })
    return dataset

def save_dataset_as_json(dataset, output_path):
    with open(output_path, 'w') as f:
        json.dump(dataset, f, indent=4)

def process_pdf_files(input_dir, output_dir, nlp_model):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        if filename.endswith(".pdf"):
            file_path = os.path.join(input_dir, filename)
            text = extract_text_from_pdf(file_path)
            paragraphs = split_into_paragraphs(text)
            dataset = create_finetuning_dataset(paragraphs, nlp_model)
            output_path = os.path.join(output_dir, filename.replace('.pdf', '.json'))
            save_dataset_as_json(dataset, output_path)

def create_knowledge_graph(output_dir):
    knowledge_graph = defaultdict(list)
    
    for filename in os.listdir(output_dir):
        if filename.endswith(".json"):
            file_path = os.path.join(output_dir, filename)
            with open(file_path, 'r') as f:
                dataset = json.load(f)
                country = filename.split('-')[1]
                for data in dataset:
                    knowledge_graph[country].append(data)
    
    with open(os.path.join(output_dir, 'knowledge_graph.json'), 'w') as f:
        json.dump(knowledge_graph, f, indent=4)

In [15]:
# Example usage:
input_directory = "documents/"
output_directory = "documents/finetuning-data/"

process_pdf_files(input_directory, output_directory, nlp)
create_knowledge_graph(output_directory)

IndexError: list index out of range